# H2O-3's AutoML vs Driverless AI (with Auto Feature Engineering)

Load the same data set in H2O-3's AutoML and Driverless AI. Compare and contrast if the Driverless AI's Feature Engineering is 'squeezing' out more performance. Evaluate the Train and Test MAE's the raw data set in AutoML and then the Feature Engineered one from Driverless back into AutoML.


In [1]:
#Import H2O libiraries
import h2o
from h2o.automl import H2OAutoML

In [2]:
#Initialize local H2O Cluster
h2o.init(min_mem_size='12G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /Users/thomasott/opt/anaconda3/envs/py37/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/9c/tqhnrz3x207bf8pzjxjhkm100000gn/T/tmp1dmeac6v
  JVM stdout: /var/folders/9c/tqhnrz3x207bf8pzjxjhkm100000gn/T/tmp1dmeac6v/h2o_thomasott_started_from_python.out
  JVM stderr: /var/folders/9c/tqhnrz3x207bf8pzjxjhkm100000gn/T/tmp1dmeac6v/h2o_thomasott_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.2
H2O cluster version age:,1 month and 4 days
H2O cluster name:,H2O_from_python_thomasott_wr2gp1
H2O cluster total nodes:,1
H2O cluster free memory:,11.50 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
#Load in Loan Loss Training Set
#Note there is a zip file in the Github folder for this CSV
my_data = h2o.import_file('./data/train_v2.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
train,test = my_data.split_frame(ratios=[.7])

In [5]:
# Set target and predictor variables
y = "loss"
x = train.col_names
x.remove(y)

#Drop the ID column
x.remove("id")

In [6]:
#Call H2O-3 AutoML. Set early stopping metric to MAE and CV Folds = 5
aml = H2OAutoML(max_models = 10, seed=1234, stopping_metric = "MAE", sort_metric = "MAE", nfolds = 5)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [7]:
#Example LeaderBoard (Note: my small CPU can only train a single model, run this on a larger machine!)
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mae,mean_residual_deviance,rmse,mse,rmsle
XGBoost_3_AutoML_20200225_143908,1.38631,17.3388,4.16399,17.3388,0.708961
XGBoost_1_AutoML_20200225_143908,1.39068,17.4487,4.17717,17.4487,0.717157
XGBoost_2_AutoML_20200225_143908,1.39627,17.5008,4.18339,17.5008,nan
StackedEnsemble_AllModels_AutoML_20200225_143908,1.40414,17.277,4.15656,17.277,0.716554
GBM_4_AutoML_20200225_143908,1.40499,17.4521,4.17757,17.4521,nan
GBM_2_AutoML_20200225_143908,1.40535,17.3962,4.17087,17.3962,nan
GBM_5_AutoML_20200225_143908,1.40583,17.3708,4.16783,17.3708,0.731379
StackedEnsemble_BestOfFamily_AutoML_20200225_143908,1.4067,17.2907,4.15821,17.2907,0.716449
GBM_3_AutoML_20200225_143908,1.4078,17.4289,4.1748,17.4289,nan
GLM_1_AutoML_20200225_143908,1.41154,17.3146,4.16109,17.3146,nan


Select the AutoML leader and apply test data set to it.

In [8]:
best_model = aml.leader.model_performance(test).mae()

Get the 'best_model' test performance.

In [9]:
best_model

1.4519806896387866

In [10]:
#Import Driverless AI transformed features from oringal training Set
#Note there is a zip file in the Github folder for this CSV
my_data_FE = h2o.import_file('./data/train_v2_FE.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [11]:
train,test = my_data_FE.split_frame(ratios=[.7])

In [12]:
#Check out the munged dataset
train

1000_ClusterDist10:f2:f230:f274:f528.0 1000_ClusterDist10:f2:f230:f274:f528.1 1000_ClusterDist10:f2:f230:f274:f528.2 1000_ClusterDist10:f2:f230:f274:f528.3 1000_ClusterDist10:f2:f230:f274:f528.4 1000_ClusterDist10:f2:f230:f274:f528.5 1000_ClusterDist10:f2:f230:f274:f528.6 1000_ClusterDist10:f2:f230:f274:f528.7 1000_ClusterDist10:f2:f230:f274:f528.8 1000_ClusterDist10:f2:f230:f274:f528.9 1023_ClusterDist8:f219:f274:f350:f397:f598:f653:f740.0 1023_ClusterDist8:f219:f274:f350:f397:f598:f653:f740.1 1023_ClusterDist8:f219:f274:f350:f397:f598:f653:f740.2 1023_ClusterDist8:f219:f274:f350:f397:f598:f653:f740.3 1023_ClusterDist8:f219:f274:f350:f397:f598:f653:f740.4 1023_ClusterDist8:f219:f274:f350:f397:f598:f653:f740.5 1023_ClusterDist8:f219:f274:f350:f397:f598:f653:f740.6 1023_ClusterDist8:f219:f274:f350:f397:f598:f653:f740.7 1032_ClusterDist6:f133:f274:f528:f620.0 1032_ClusterDist6:f133:f274:f528:f620.1 1032_ClusterDist6:f133:f274:f528:f620.2 1032_ClusterDist6:f133:f274:f528:f620.3 1032_ClusterDist6:f133:f274:f528:f620.4 1032_ClusterDist6:f133:f274:f528:f620.5 1048_TruncSVD:f221:f274:f333:f525:f528.1 1048_TruncSVD:f221:f274:f333:f525:f528.2 1061_ClusterDist8:f274:f528:f533:f648.0 1061_ClusterDist8:f274:f528:f533:f648.1 1061_ClusterDist8:f274:f528:f533:f648.2 1061_ClusterDist8:f274:f528:f533:f648.3 1061_ClusterDist8:f274:f528:f533:f648.4 1061_ClusterDist8:f274:f528:f533:f648.5 1061_ClusterDist8:f274:f528:f533:f648.6 1061_ClusterDist8:f274:f528:f533:f648.7 1074_ClusterDist10:f133:f272:f378:f527.0 1074_ClusterDist10:f133:f272:f378:f527.1 1074_ClusterDist10:f133:f272:f378:f527.2 1074_ClusterDist10:f133:f272:f378:f527.3 1074_ClusterDist10:f133:f272:f378:f527.4 1074_ClusterDist10:f133:f272:f378:f527.5 1074_ClusterDist10:f133:f272:f378:f527.7 1074_ClusterDist10:f133:f272:f378:f527.8 1074_ClusterDist10:f133:f272:f378:f527.9 1083_ClusterDist8:f2:f274:f383:f527:f528.0 1083_ClusterDist8:f2:f274:f383:f527:f528.1 1083_ClusterDist8:f2:f274:f383:f527:f528.2 1083_ClusterDist8:f2:f274:f383:f527:f528.3 1083_ClusterDist8:f2:f274:f383:f527:f528.5 1083_ClusterDist8:f2:f274:f383:f527:f528.6 1083_ClusterDist8:f2:f274:f383:f527:f528.7 1091_InteractionSub:f332:f458 1098_ClusterDist10:f2:f222:f252:f274:f528:f535.0 1098_ClusterDist10:f2:f222:f252:f274:f528:f535.1 1098_ClusterDist10:f2:f222:f252:f274:f528:f535.2 1098_ClusterDist10:f2:f222:f252:f274:f528:f535.3 1098_ClusterDist10:f2:f222:f252:f274:f528:f535.4 1098_ClusterDist10:f2:f222:f252:f274:f528:f535.5 1098_ClusterDist10:f2:f222:f252:f274:f528:f535.6 1098_ClusterDist10:f2:f222:f252:f274:f528:f535.7 1098_ClusterDist10:f2:f222:f252:f274:f528:f535.8 1098_ClusterDist10:f2:f222:f252:f274:f528:f535.9 1102_ClusterDist10:f108:f2:f527.0 1102_ClusterDist10:f108:f2:f527.1 1102_ClusterDist10:f108:f2:f527.2 1102_ClusterDist10:f108:f2:f527.3 1102_ClusterDist10:f108:f2:f527.4 1102_ClusterDist10:f108:f2:f527.5 1102_ClusterDist10:f108:f2:f527.6 1102_ClusterDist10:f108:f2:f527.7 1102_ClusterDist10:f108:f2:f527.8 1102_ClusterDist10:f108:f2:f527.9 1108_ClusterDist10:f16:f272:f274:f332:f527:f528.0 1108_ClusterDist10:f16:f272:f274:f332:f527:f528.1 1108_ClusterDist10:f16:f272:f274:f332:f527:f528.2 1108_ClusterDist10:f16:f272:f274:f332:f527:f528.3 1108_ClusterDist10:f16:f272:f274:f332:f527:f528.4 1108_ClusterDist10:f16:f272:f274:f332:f527:f528.5 1108_ClusterDist10:f16:f272:f274:f332:f527:f528.6 1108_ClusterDist10:f16:f272:f274:f332:f527:f528.7 1108_ClusterDist10:f16:f272:f274:f332:f527:f528.8 1108_ClusterDist10:f16:f272:f274:f332:f527:f528.9 110_f201 1119_ClusterDist4:f221:f24:f528.0 1119_ClusterDist4:f221:f24:f528.1 1119_ClusterDist4:f221:f24:f528.2 1119_ClusterDist4:f221:f24:f528.3 1130_TruncSVD:f2:f221:f274:f527:f528:f6:f601.1 1130_TruncSVD:f2:f221:f274:f527:f528:f6:f601.2 1137_ClusterDist6:f260:f527.0 1137_ClusterDist6:f260:f527.1 1137_ClusterDist6:f260:f527.2 1137_ClusterDist6:f260:f527.3 1137_ClusterDist6:f260:f527.4 1137_ClusterDist6:f260:f527.5 1140_ClusterDist10:f147:f516:f527.0 1140_ClusterDist10:f147:f516:f527.1 1140_Cl

In [13]:
# Set target and predictor variables
y = "loss"
x = train.col_names
x.remove(y)

In [14]:
#Run another AutoML with munged data set
aml_FE = H2OAutoML(max_models = 15, seed=1234, stopping_metric = "MAE", sort_metric = "MAE", nfolds = 5)
aml_FE.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [15]:
#check out top models (note the munged data set makes training faster)
lb = aml_FE.leaderboard
lb.head(rows=lb.nrows)

model_id,mae,mean_residual_deviance,rmse,mse,rmsle
XGBoost_1_AutoML_20200225_164224,0.610655,10.1747,3.18978,10.1747,nan
XGBoost_grid__1_AutoML_20200225_164224_model_1,0.618415,10.4849,3.23804,10.4849,nan
XGBoost_2_AutoML_20200225_164224,0.622748,10.2142,3.19597,10.2142,nan
GBM_3_AutoML_20200225_164224,0.626351,10.1268,3.18226,10.1268,nan
GBM_4_AutoML_20200225_164224,0.629277,10.1393,3.18422,10.1393,nan
XRT_1_AutoML_20200225_164224,0.630574,10.624,3.25945,10.624,0.347487
XGBoost_3_AutoML_20200225_164224,0.631228,10.4252,3.22881,10.4252,nan
GBM_2_AutoML_20200225_164224,0.631979,10.2321,3.19877,10.2321,nan
DRF_1_AutoML_20200225_164224,0.632404,10.6047,3.25649,10.6047,0.349792
XGBoost_grid__1_AutoML_20200225_164224_model_2,0.641982,10.4365,3.23056,10.4365,nan


In [16]:
best_model_FE = aml_FE.leader.model_performance(test).mae()

In [17]:
best_model_FE

0.5969730445882833

In [18]:
h2o.cluster().shutdown()

H2O session _sid_97f1 closed.
